In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/')

In [ ]:
!ls "Intent classification"

test.csv  test_labels.csv  train.csv


In [ ]:
!cp "Intent classification"/* /content/

In [ ]:
train_data = pd.read_csv('/content/train.csv')
test_data = pd.read_csv('/content/test.csv')

In [ ]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [ ]:
print(train_data.shape, test_data.shape)

(159571, 8) (153164, 2)


In [ ]:
train_data.comment_text = [text.lower() for text in train_data.comment_text]
test_data.comment_text = [text.lower() for text in test_data.comment_text]

In [ ]:
from nltk.tokenize import word_tokenize
train_data.comment_text = [word_tokenize(text) for text in train_data.comment_text]
test_data.comment_text = [word_tokenize(text) for text in test_data.comment_text]

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
train_data.comment_text = [word for word in train_data.comment_text if not word in stop]
test_data.comment_text = [word for word in test_data.comment_text if not word in stop]

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stem = PorterStemmer()
lem = WordNetLemmatizer()

def stem_lem(data):
    lemmatized = []
    for text in data:
        final = []
        for word in text:
            #Check if the recursion depth does not exceed
            if len(word) < 990:
                stem.stem(word)
                final.append(lem.lemmatize(word))
        lemmatized.append(final)
    return lemmatized

train_data.comment_text = stem_lem(train_data.comment_text)
test_data.comment_text = stem_lem(test_data.comment_text)

In [ ]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"[explanation, why, the, edits, made, under, my...",0,0,0,0,0,0
1,000103f0d9cfb60f,"[d'aww, !, he, match, this, background, colour...",0,0,0,0,0,0
2,000113f07ec002fd,"[hey, man, ,, i, 'm, really, not, trying, to, ...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"[``, more, i, ca, n't, make, any, real, sugges...",0,0,0,0,0,0
4,0001d958c54c6e35,"[you, ,, sir, ,, are, my, hero, ., any, chance...",0,0,0,0,0,0


In [ ]:
#train_data.to_csv('/content/preprocessed_train_data.csv', index=False)
#test_data.to_csv('/content/preprocessed_test_data.csv', index=False)

In [ ]:
# Move files to destination folder
#!cp /content/preprocessed_train_data.csv '/content/drive/My Drive/Intent classification/'
#!cp /content/preprocessed_test_data.csv '/content/drive/My Drive/Intent classification/'

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, Bidirectional, LSTM
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import Sequential
from keras.preprocessing import text
#from keras.utils import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train = train_data["comment_text"]
y_train = train_data[["toxic", 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
X_test = test_data["comment_text"]

tokenizer = text.Tokenizer()

tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_l = 100

X_train = pad_sequences(X_train, maxlen = max_l)
X_test = pad_sequences(X_test, maxlen = max_l)

In [ ]:
X_train.shape

(159571, 100)

In [ ]:
MAX_FEATURES = max(tokenizer.word_index.values())
print(MAX_FEATURES)

473612


In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow  as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

Device: grpc://10.2.26.2:8470


Number of replicas: 8


In [ ]:
with strategy.scope():
  model = Sequential()
  model.add(Embedding(MAX_FEATURES + 1, 50, input_shape = (100, )))
  model.add(Bidirectional(LSTM(50, dropout=0.1)))
  model.add(Dense(50, activation = 'relu'))
  model.add(Dropout(0.1))
  model.add(Dense(6, activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics= [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 50)           23680650  
                                                                 
 bidirectional_4 (Bidirectio  (None, 100)              40400     
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dropout_4 (Dropout)         (None, 50)                0         
                                                                 
 dense_9 (Dense)             (None, 6)                 306       
                                                                 
Total params: 23,726,406
Trainable params: 23,726,406
Non-trainable params: 0
__________________________________________

In [ ]:
history = model.fit(X_train, y_train, epochs = 5, validation_split = 0.1, batch_size = 32)

Epoch 1/5
4488/4488 [==============================] - 195s 40ms/step - loss: 0.0673 - accuracy: 0.9781 - precision: 0.7827 - recall: 0.5570 - val_loss: 0.0517 - val_accuracy: 0.9816 - val_precision: 0.8143 - val_recall: 0.6550
Epoch 2/5
4488/4488 [==============================] - 176s 39ms/step - loss: 0.0449 - accuracy: 0.9831 - precision: 0.8242 - recall: 0.6834 - val_loss: 0.0500 - val_accuracy: 0.9817 - val_precision: 0.8100 - val_recall: 0.6656
Epoch 3/5
4488/4488 [==============================] - 176s 39ms/step - loss: 0.0379 - accuracy: 0.9852 - precision: 0.8412 - recall: 0.7329 - val_loss: 0.0514 - val_accuracy: 0.9814 - val_precision: 0.8051 - val_recall: 0.6609
Epoch 4/5
 597/4488 [==>...........................] - ETA: 2:26 - loss: 0.0323 - accuracy: 0.9873 - precision: 0.8643 - recall: 0.7702

KeyboardInterrupt: 

In [ ]:
model.save('/content/intent_model2.h5')

In [ ]:
from google.colab import files

# Specify the file path you want to download
file_path = '/content/intent_model2.h5'

# Download the file
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save_weights('/content/intent_model_weights2.h5')

In [ ]:
from google.colab import files

# Specify the file path you want to download
file_path = '/content/intent_model_weights2.h5'

# Download the file
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle
# Convert the Keras model to a dictionary representation
model_dict = model.to_json()

# Save the model dictionary as a pickle file
with open('/content/intent_model2.pkl', 'wb') as f:
    pickle.dump(model_dict, f)

In [ ]:
from google.colab import files

# Specify the file path you want to download
file_path = '/content/intent_model2.pkl'

# Download the file
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
predictions = model.predict(X_test)

4787/4787 [==============================] - 60s 12ms/step


In [ ]:
df = pd.DataFrame({'comment_text': test_data['comment_text']})
cols = train_data[["toxic", 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
predictions_df = pd.concat([df, pd.DataFrame(predictions, columns = cols.columns)], axis=1)
predictions_df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"[yo, bitch, ja, rule, is, more, succesful, the...",0.994968,3.433412e-01,9.777049e-01,5.353826e-02,8.690145e-01,1.559326e-01
1,"[==, from, rfc, ==, the, title, is, fine, a, i...",0.000066,0.000000e+00,2.384186e-07,5.960464e-08,9.238720e-07,2.384186e-07
2,"[``, ==, source, ==, *, zawe, ashton, on, lapl...",0.014091,2.831221e-06,1.111746e-03,1.478195e-05,5.380213e-04,2.250075e-05
3,"[:, if, you, have, a, look, back, at, the, sou...",0.000305,1.788139e-07,3.069639e-06,3.874302e-07,3.069639e-06,1.281500e-06
4,"[i, do, n't, anonymously, edit, article, at, a...",0.001383,1.043081e-06,5.990267e-05,7.390976e-06,6.258488e-05,4.261732e-06


In [ ]:
y_pred = model.predict(X_train)

4987/4987 [==============================] - 72s 14ms/step


In [ ]:
y_pred = (y_pred > 0.315)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89     15294
           1       0.50      0.58      0.54      1595
           2       0.84      0.90      0.87      8449
           3       0.00      0.00      0.00       478
           4       0.74      0.85      0.79      7877
           5       0.53      0.26      0.35      1405

   micro avg       0.80      0.85      0.83     35098
   macro avg       0.58      0.59      0.57     35098
weighted avg       0.79      0.85      0.81     35098
 samples avg       0.08      0.08      0.08     35098



In [ ]:
import tensorflow as tf
from tqdm import tqdm

import transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel

In [ ]:
# Using BERT Auto-Tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
MAX_LEN = 128
def tokenize(data,max_len = MAX_LEN):
    input_ids = list()
    attention_mask = list()
    for i in tqdm(range(len(data))):
        encoded = tokenizer.encode_plus(data[i],
                                        add_special_tokens = True,
                                        max_length = MAX_LEN,
                                        is_split_into_words=True,
                                        return_attention_mask=True,
                                        padding = 'max_length',
                                        truncation=True,return_tensors = 'np')


        input_ids.append(encoded['input_ids'])
        attention_mask.append(encoded['attention_mask'])
    return np.vstack(input_ids),np.vstack(attention_mask)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_data = pd.read_csv('/content/preprocessed_train_data.csv')

In [ ]:
X = train_data["comment_text"].values
y = train_data[["toxic", 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

In [ ]:
import ast
X = [ast.literal_eval(s) for s in X]
X = np.asarray(X)

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,random_state=42,test_size=0.1)
X_train.shape,X_val.shape,y_train.shape,y_val.shape

((143613,), (15958,), (143613, 6), (15958, 6))

In [ ]:
input_ids, attention_mask = tokenize(X_train, max_len=MAX_LEN)
val_input_ids, val_attention_mask = tokenize(X_val, max_len=MAX_LEN)

100%|██████████| 15958/15958 [00:18<00:00, 862.90it/s] 


In [ ]:
input_ids[0]

array([  101, 25506,   999,   999,   999,   999,   999,  2023,  3796,
        2003, 25506,  1998,  4895,  1011, 15536,  3211,   999, 10294,
       27555,  2003,  2055,  1037, 23458,  2051,  2062,  2590,  1010,
       11419,  1998,  4069,  2084,  2012,  2560,  5585,  1003,  1997,
        2035,  1996,  3720,  3395,  2006,  2023,  8257,  1997,  1037,
        2609,  1012,  2339,  1010,  2821,  2339,  1010,  2024, 16948,
        1005,  1055,  8911,  2061, 13847,  1037,  1008,  1008,  1008,
        1011,  2128,  6528,  6024,  1998,  2031,  2107,  2019, 14092,
        3012,  3375,  1998, 13072, 13059,  1029,  1029,  1029,  2079,
        2035,  1997,  2017,  2031,  2000, 10083,  1996,  4569,  1998,
       12024,  1997,  1996,  2717,  1997,  1057,  2266,  2074,  2138,
        2017,  2342,  2000,  2514,  3928,  1998,  2590,  2061,  9652,
        1029,  1029,  2024,  2035,  1997,  2017,  2082,  9388,  2213,
       13850,  2828,  1999, 12121, 14329,  2007,  4317,  3221,  1010,
        4452,   102]

In [ ]:
# TEST: Checking Padding and Truncation length's
was = list()
for i in range(len(input_ids)):
    was.append(len(input_ids[i]))
set(was)

{128}

In [ ]:
def create_model(bert_model, max_len=MAX_LEN):
    input_ids = tf.keras.Input(shape=(max_len,), dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,), dtype='int32')
    bert_output = bert_model(input_ids, attention_mask=attention_masks, return_dict=True)
    embedding = tf.keras.layers.Dropout(0.3)(bert_output["pooler_output"])
    output = tf.keras.layers.Dense(50, activation='relu')(embedding)
    output = tf.keras.layers.Dropout(0.1)(output)
    output = tf.keras.layers.Dense(6, activation='sigmoid')(output)
    model = tf.keras.models.Model(inputs=[input_ids, attention_masks], outputs=[output])
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss="binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
    ])
    return model

In [ ]:
import tensorflow  as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

Device: grpc://10.2.26.2:8470


Number of replicas: 8


In [ ]:
with strategy.scope():
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    model = create_model(bert_model,MAX_LEN)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                           

In [ ]:
y_val

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 1]])

In [ ]:
history_bert = model.fit([input_ids,attention_mask],y_train,validation_data = ([val_input_ids,val_attention_mask],y_val),epochs = 2,batch_size = 32, verbose = True)

Epoch 1/2
4488/4488 [==============================] - 494s 89ms/step - loss: 0.1498 - accuracy: 0.9630 - precision: 0.0421 - recall: 3.7904e-04 - val_loss: 0.1417 - val_accuracy: 0.9641 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
4488/4488 [==============================] - 389s 87ms/step - loss: 0.1454 - accuracy: 0.9633 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.1402 - val_accuracy: 0.9641 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00


In [ ]:
test_data = pd.read_csv('/content/preprocessed_test_data.csv')

In [ ]:
X_test = test_data["comment_text"].values

In [ ]:
import ast
X_test = [ast.literal_eval(s) for s in X_test]
X_test = np.asarray(X_test)

In [ ]:
test_input_ids, test_attention_mask = tokenize(X_test, max_len=MAX_LEN)

100%|██████████| 153164/153164 [02:57<00:00, 863.39it/s]


In [ ]:
predictions = model.predict([test_input_ids, test_attention_mask])

4787/4787 [==============================] - 115s 23ms/step


In [ ]:
df = pd.DataFrame({'comment_text': test_data['comment_text']})
cols = train_data[["toxic", 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]
predictions_df = pd.concat([df, pd.DataFrame(predictions, columns = cols.columns)], axis=1)
predictions_df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"['yo', 'bitch', 'ja', 'rule', 'is', 'more', 's...",0.084679,0.004622,0.04949,0.001575,0.042893,0.002603
1,"['==', 'from', 'rfc', '==', 'the', 'title', 'i...",0.084679,0.004622,0.04949,0.001575,0.042893,0.002603
2,"['``', '==', 'source', '==', '*', 'zawe', 'ash...",0.084679,0.004622,0.04949,0.001575,0.042893,0.002603
3,"[':', 'if', 'you', 'have', 'a', 'look', 'back'...",0.084679,0.004622,0.04949,0.001575,0.042893,0.002603
4,"['i', 'do', ""n't"", 'anonymously', 'edit', 'art...",0.084679,0.004622,0.04949,0.001575,0.042893,0.002603


In [ ]:
overall_input_ids, overall_attention_mask = tokenize(X, max_len=MAX_LEN)

100%|██████████| 159571/159571 [03:04<00:00, 862.88it/s] 


In [ ]:
overall_pred = model.predict([overall_input_ids, overall_attention_mask])

4987/4987 [==============================] - 110s 22ms/step


In [ ]:
overall_pred_temp = (overall_pred > 0.5)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y, overall_pred_temp))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     15294
           1       0.00      0.00      0.00      1595
           2       0.00      0.00      0.00      8449
           3       0.00      0.00      0.00       478
           4       0.00      0.00      0.00      7877
           5       0.00      0.00      0.00      1405

   micro avg       0.00      0.00      0.00     35098
   macro avg       0.00      0.00      0.00     35098
weighted avg       0.00      0.00      0.00     35098
 samples avg       0.00      0.00      0.00     35098

